In [93]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_drug.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/test_features.csv


In [94]:
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import model_selection

import random
random.seed(123)

In [95]:
train_features_df = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
print(train_features_df.shape)
train_features_df

(23814, 876)


,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
23810,id_fffb70c0c,trt_cp,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
23811,id_fffc1c3f4,ctl_vehicle,48,D2,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,...,0.5409,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808
23812,id_fffcb9e7c,trt_cp,24,D1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,...,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740


In [96]:
train_targets_scored_df = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
print(train_targets_scored_df.shape)
train_targets_scored_df

(23814, 207)


,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23810,id_fffb70c0c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23811,id_fffc1c3f4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23812,id_fffcb9e7c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
train_len, val_len = train_test_split([i for i in range(0, 23814)], test_size=0.2)
len(train_len), len(val_len)

(19051, 4763)

In [98]:
train_features = train_features_df.iloc[train_len, 1:]
val_features = train_features_df.iloc[val_len, 1:]

train_labels = train_targets_scored_df.iloc[train_len, 1:]
val_labels = train_targets_scored_df.iloc[val_len, 1:]

len(train_features), len(val_features), len(train_labels), len(val_labels)

(19051, 4763, 19051, 4763)

In [99]:
train_features.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
9197,trt_cp,48,D2,-0.2763,0.5609,-0.2262,0.3724,0.2966,-0.0864,-0.3850,...,-0.0129,0.6536,-0.1897,0.2048,0.7780,-0.1461,0.5883,-0.3812,-1.3970,1.2730
9206,trt_cp,48,D2,-0.2873,0.7694,-0.3657,0.0496,0.0267,-0.4707,-0.9181,...,0.0907,0.5309,0.1664,0.9549,0.5475,-0.6463,1.0160,-0.6930,0.6531,0.8857
16880,trt_cp,72,D1,1.6050,-1.4360,0.0841,0.2057,1.1100,0.7012,0.7116,...,-1.2520,0.1955,-0.7292,-0.5513,-0.4246,-1.4610,-1.4480,-0.2435,-1.3240,0.4754
20841,trt_cp,72,D2,-1.1880,-0.2347,0.0395,0.1852,0.1268,-1.6040,-1.6180,...,-3.2770,-0.7388,-8.1940,-1.4670,-3.0840,-3.1390,-4.1110,-6.3090,-1.8640,-0.6673
10220,trt_cp,72,D2,0.1189,-1.4850,-3.2760,-0.2930,-0.3259,-0.2495,-1.0960,...,-1.2740,-0.6922,-0.4150,-0.2517,-0.0898,0.0690,0.2369,-1.0210,-0.2065,0.5580


In [100]:
train_labels.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
9197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9206,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16880,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20841,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
val_features.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
15160,trt_cp,48,D2,0.1629,-0.2843,-0.2171,0.0790,0.2534,0.4654,-0.4185,...,0.3139,-0.5475,0.6388,-0.6749,-0.1913,0.1807,1.0350,0.3843,0.4945,0.0798
19796,trt_cp,48,D2,-0.0775,0.0352,-0.5327,-1.2530,0.4514,0.3047,0.9752,...,-0.0418,1.1380,-0.4182,-0.4890,0.4055,-0.1543,0.1897,-0.4264,-0.3671,-0.1982
9040,trt_cp,48,D1,-0.5487,-0.0030,0.4228,0.2290,-0.5690,1.8890,0.5108,...,-0.0632,0.5525,-0.3747,0.2370,-0.2374,-0.0811,0.2660,-0.4976,-0.2824,-1.0590
5610,trt_cp,72,D1,0.1381,-0.1226,-0.1169,0.2203,0.7432,0.4223,1.1830,...,0.7220,0.6714,0.3382,0.2513,-0.4646,-0.7745,-0.8201,1.4510,-0.1919,0.5287
23103,trt_cp,72,D2,0.0664,-0.6415,0.1397,-0.0413,-0.0912,-3.0310,-1.4770,...,-0.1084,0.6299,-0.6726,-0.0622,-0.4475,0.0139,-1.2540,-0.3071,-2.2440,0.2653


In [102]:
val_labels.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
15160,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19796,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9040,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5610,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
23103,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [103]:
## handling cp_type

cp_type_dict = {
    "trt_cp": 0,
    "ctl_vehicle": 1
}

train_features['cp_type'] = train_features.cp_type.map(cp_type_dict)
val_features['cp_type'] = val_features.cp_type.map(cp_type_dict)

In [104]:
## handling cp_dose

cp_dose_dict = {
    "D1": 1,
    "D2": 2
}

train_features["cp_dose"] = train_features.cp_dose.map(cp_dose_dict)
val_features["cp_dose"] = val_features.cp_dose.map(cp_dose_dict)

In [105]:
## handling cp_time 

cp_time_dict = {
    24:1, 
    48:2,
    72:3
}

train_features['cp_time'] = train_features.cp_time.map(cp_time_dict)
val_features['cp_time'] = val_features.cp_time.map(cp_time_dict)

In [106]:
## Standard scaling the continuous columns 

continuous_cols = [col for col in list(train_features_df.columns) if col not in ["cp_time", "cp_dose", "cp_time", "sig_id"]]

train_continuous_cols_df = train_features[continuous_cols].copy()
standard_scaler_object = StandardScaler().fit(train_continuous_cols_df.values)
train_continuous_cols_df = standard_scaler_object.transform(train_continuous_cols_df.values)

val_continuous_cols_df = val_features[continuous_cols].copy()
val_continuous_cols_df = standard_scaler_object.transform(val_continuous_cols_df.values)

# adding scaled values to original dataframe 
train_features[continuous_cols] = train_continuous_cols_df
val_features[continuous_cols] = val_continuous_cols_df

In [107]:
all_categories = list(train_labels.columns)
len(all_categories)

206

In [108]:
lr_model_dict = {}

for category in tqdm(all_categories):
    # Training logistic regression model on training data
    logistic_model = LogisticRegression(max_iter=5000)
    logistic_model.fit(train_features, train_labels[category])
    
    # saving model
    lr_model_dict[category] = logistic_model
    

In [ ]:
# rf_model_dict = {}

# for category in tqdm(all_categories):
#     # Training random forest model on training data
#     rf_model = RandomForestClassifier(n_estimators=100)
#     rf_model.fit(train_features, train_labels[category])
    
#     # saving model
#     rf_model_dict[category] = rf_model

In [ ]:
# dt_model_dict = {}

# for category in tqdm(all_categories):
#     # Training decision tree model on training data
#     dt_model = DecisionTreeClassifier(random_state=0)
#     dt_model.fit(train_features, train_labels[category])
    
#     # saving model
#     dt_model_dict[category] = dt_model

In [ ]:
ovr_model_dict = {}

for category in tqdm(all_categories):
    # Training OneVsRest classifier model on training data
    lr_pipeline = Pipeline([('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),])
    lr_pipeline.fit(train_features, train_labels[category])
    
    # saving model 
    ovr_model_dict[category] = lr_pipeline 

In [ ]:
etc_model_dict = {}

for category in tqdm(all_categories):
    # Training ExtraTreeClassifier model on training data 
    etc_model = ExtraTreesClassifier(n_estimators=100, random_state=0)
    etc_model.fit(train_features, train_labels[category])
    
    # saving model
    etc_model_dict[category] = etc_model

In [ ]:
def calculate_score_lr(lr_model_dict, val_features, val_labels, all_categories):
    log_loss_per_category = []
    for category in tqdm(all_categories):
        extra_model = lr_model_dict[category]
        category_probs = extra_model.predict_proba(val_features)
        
        log_loss_per_category.append(log_loss(val_labels[category], category_probs, labels=[0,1]))
        
    return float(sum(log_loss_per_category))/len(log_loss_per_category)

In [ ]:
lr_val_score = calculate_score_lr(lr_model_dict, val_features, val_labels, all_categories)
print("The log loss score for the logistic regression model is : ", lr_val_score)

In [ ]:
# def calculate_score_rf(rf_model_dict, val_features, val_labels, all_categories):
#     log_loss_per_category = []
#     for category in tqdm(all_categories):
#         extra_model = rf_model_dict[category]
#         category_probs = extra_model.predict_proba(val_features)
        
#         log_loss_per_category.append(log_loss(val_labels[category], category_probs, labels=[0,1]))
        
#     return float(sum(log_loss_per_category))/len(log_loss_per_category)

In [ ]:
# rf_val_score = calculate_score_rf(rf_model_dict, val_features, val_labels, all_categories)
# rf_val_score

In [ ]:
def calculate_score_ovr(ovr_model_dict, val_features, val_labels, all_categories):
    log_loss_per_category = []
    for category in tqdm(all_categories):
        extra_model = ovr_model_dict[category]
        category_probs = extra_model.predict_proba(val_features)
        
        log_loss_per_category.append(log_loss(val_labels[category], category_probs, labels=[0,1]))
        
    return float(sum(log_loss_per_category))/len(log_loss_per_category)

In [ ]:
ovr_val_score = calculate_score_ovr(ovr_model_dict, val_features, val_labels, all_categories)
print("The log loss score for the OneVsRest classifier model is: ", ovr_val_score)

In [ ]:
def calculate_score_etc(etc_model_dict, val_features, val_labels, all_categories):
    log_loss_per_category = []
    for category in tqdm(all_categories):
        extra_model = etc_model_dict[category]
        category_probs = extra_model.predict_proba(val_features)
        
        log_loss_per_category.append(log_loss(val_labels[category], category_probs, labels=[0,1]))
        
    return float(sum(log_loss_per_category))/len(log_loss_per_category)

In [ ]:
etc_val_score = calculate_score_etc(etc_model_dict, val_features, val_labels, all_categories)
print("The log loss score for the ExtraTrees classifier model is : ", etc_val_score)

In [ ]:
# test_features_df = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")
# print(test_features_df.shape)
# test_features_df.head()

In [ ]:
# test_features_df["cp_type"] = test_features_df.cp_type.map(cp_type_dict)
# test_features_df["cp_dose"] = test_features_df.cp_dose.map(cp_dose_dict)
# test_features_df["cp_time"] = test_features_df.cp_time.map(cp_time_dict)

In [ ]:
# test_continuous_columns_df = test_features_df[continuous_cols].copy()
# test_continuous_columns_df = standard_scaler_object.transform(test_continuous_columns_df.values)
# test_features_df[continuous_cols] = test_continuous_columns_df

In [ ]:
# predictions_df = pd.DataFrame()
# predictions_df["sig_id"] = test_features_df.sig_id
# for category in tqdm(all_categories):
#     predictions_df[category] = lr_model_dict[category].predict_proba(test_features_df.iloc[:, 1:])[:, 1]

In [ ]:
# predictions_df = predictions_df.round(1)
# predictions_df.head()

In [ ]:
# test_features_df.head()

In [ ]:
# predictions_df.shape

In [ ]:
# predictions_df.head()

In [ ]:
# predictions_df.to_csv("submission.csv", index=False)